<a href="https://colab.research.google.com/github/HuyenNguyenHelen/LING-5412/blob/main/Assignment3_LinearModel.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [30]:
# Importing libraries that will be used 
import numpy as np
import tarfile
import glob
import re
import pandas as pd
#from nltk.corpus import stopwords
import nltk
nltk.download('wordnet')
import sklearn
from sklearn.model_selection import GridSearchCV
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.linear_model import SGDClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


# Loading the dataset

In [31]:
!wget http://www.cs.cornell.edu/people/pabo/movie-review-data/scale_data.tar.gz

--2021-10-12 05:45:42--  http://www.cs.cornell.edu/people/pabo/movie-review-data/scale_data.tar.gz
Resolving www.cs.cornell.edu (www.cs.cornell.edu)... 132.236.207.36
Connecting to www.cs.cornell.edu (www.cs.cornell.edu)|132.236.207.36|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 4029756 (3.8M) [application/x-gzip]
Saving to: ‘scale_data.tar.gz.3’

scale_data.tar.gz.3 100%[===================>]   3.84M  16.0MB/s    in 0.2s    

2021-10-12 05:45:43 (16.0 MB/s) - ‘scale_data.tar.gz.3’ saved [4029756/4029756]



In [32]:
# Untar the dataset
my_tar = tarfile.open('/content/scale_data.tar.gz')
my_tar.extractall('/content/') 
my_tar.close()


In [33]:
id, review, rating, cl_3, cl_4 = [],[],[],[], []
for root in glob.glob ('/content/scaledata/*'):
  for path in glob.glob(root+'/*'): 
      fo = open(path)
      doc = fo.read()
      if 'id' in path:
        id.extend(doc.split('\n'))
      elif 'subj' in path:
          review.extend(doc.split('\n'))
      elif 'rating' in path:
        rating.extend(doc.split('\n'))
      elif '3class' in path:
        cl_3.extend(doc.split('\n'))
      else:
        cl_4.extend(doc.split('\n'))
      # print(path,'\n')
      # print(doc.split('\n')[:10])
print (len(id), len(review), len(rating), len(cl_3), len(cl_4))

data = pd.DataFrame(zip(id, review, rating, cl_3, cl_4), columns = ['id', 'review', 'rating', '3class', '4class'])
data = data[['review', '3class', '4class']].dropna()
data

5010 5010 5010 5010 5010


,review,3class,4class
0,i didn't expect ghost in the machine to be thi...,0,0
1,"admittedly , with a title like the mangler , y...",0,0
2,"cast : christopher lambert , natasha henstridg...",0,0
3,"inevitably , someone is going to ask me why i ...",0,0
4,ernest : the one-joke concept that refuses to ...,0,0
...,...,...,...
5005,nicolas roeg's mesmerizing 1971 film walkabout...,2,3
5006,the movie air force one should require a docto...,2,3
5007,""" well , jones , at least you haven't forgotte...",2,3
5008,in a time of bloated productions where special...,2,3


# Pre-processing the data

In [34]:
data['3class'].value_counts()

1    1915
2    1894
0    1197
        4
Name: 3class, dtype: int64

In [35]:
data = data[data['3class']!='']


In [36]:
# Spliting the dataset for training and testing
X_train, X_test, y_train, y_test = train_test_split (data['review'], data['3class'] , train_size = 0.8, random_state = 42, shuffle = True, stratify=data['3class'])
print ('Shapes of X_train, y_train: ', X_train.shape, y_train.shape)
print ('Shapes of X_test, y_test: ', X_test.shape, y_test.shape)

Shapes of X_train, y_train:  (4004,) (4004,)
Shapes of X_test, y_test:  (1002,) (1002,)


In [37]:
# Vectorizing the documents
vectorizer = CountVectorizer(binary = True)
X_train = vectorizer.fit_transform(X_train.to_list())
X_test = vectorizer.transform(X_test.to_list())
print ('Shapes of X_train, y_train: ', X_train.shape, y_train.shape)
print ('Shapes of X_test, y_test: ', X_test.shape, y_test.shape)

Shapes of X_train, y_train:  (4004, 38609) (4004,)
Shapes of X_test, y_test:  (1002, 38609) (1002,)


# Linear models

### Default

In [38]:
def printing_eval_scores (y_true, y_pred, report=''):
  accuracy = sklearn.metrics.accuracy_score(y_true, y_pred)
  precision = sklearn.metrics.precision_score(y_true, y_pred, average = 'weighted')
  recall = sklearn.metrics.recall_score(y_true, y_pred, average = 'weighted')
  f1 = sklearn.metrics.f1_score(y_true, y_pred, average = 'weighted')
  print('accuracy score: {:.3f}'.format(accuracy))
  print('precision score: {:.3f}'.format(precision))
  print('recall score: {:.3f}'.format(recall))
  print('F1 score: {:.3f}'.format(f1))
  if report is True:
    print(classification_report(y_true, y_pred))
  else:
    pass
  return accuracy, precision, recall, f1

In [39]:

def training (penalty, max_iter): #penalty="l2", max_iter=100
  SGD = SGDClassifier( penalty=penalty, max_iter=max_iter, shuffle=True).fit(X_train, y_train)
  y_pred = SGD.predict(X_test)
  printing_eval_scores (y_test, y_pred, report=True)

penalties = ['l1','l2']
for penalty in penalties:
  print('\n-------training linear model with penalty = {}'.format(penalty ))
  training (penalty='l2', max_iter=100)


-------training linear model with penalty = l1
accuracy score: 0.650
precision score: 0.658
recall score: 0.650
F1 score: 0.649
              precision    recall  f1-score   support

           0       0.68      0.50      0.57       240
           1       0.57      0.67      0.61       383
           2       0.73      0.73      0.73       379

    accuracy                           0.65      1002
   macro avg       0.66      0.63      0.64      1002
weighted avg       0.66      0.65      0.65      1002


-------training linear model with penalty = l2
accuracy score: 0.647
precision score: 0.648
recall score: 0.647
F1 score: 0.647
              precision    recall  f1-score   support

           0       0.63      0.62      0.63       240
           1       0.57      0.59      0.58       383
           2       0.74      0.72      0.73       379

    accuracy                           0.65      1002
   macro avg       0.65      0.64      0.65      1002
weighted avg       0.65      0.65  

Penalty = L2, and loss = log (logistic regression model) yeild the best performance on F1 score (0.657)

### Searching the best parameter

In [40]:
# Set the parameters by cross-validation
tuned_parameters = [{'alpha': [i for i in np.arange (0.0001, 0.01, 0.001 )], 'loss': ['log','hinge', 'perceptron']}]
score = 'f1'

print("# Searching hyper-parameters for %s" % score)
clf = GridSearchCV(SGDClassifier(shuffle=True), tuned_parameters, scoring='%s_macro' % score, cv = 5, refit = True, n_jobs = -1 )
clf.fit(X_train, y_train)
print('Best parameters set found on development set: {}'.format(clf.best_params_))
print("Grid scores on development set:")
means = clf.cv_results_['mean_test_score']
stds = clf.cv_results_['std_test_score']

y_pred =  clf.predict(X_test)
printing_eval_scores (y_test, y_pred, report=True)



# Tuning hyper-parameters for f1
Best parameters set found on development set: {'alpha': 0.0091, 'loss': 'log'}
Grid scores on development set:
accuracy score: 0.678
precision score: 0.684
recall score: 0.678
F1 score: 0.679
              precision    recall  f1-score   support

           0       0.67      0.60      0.63       240
           1       0.60      0.68      0.64       383
           2       0.78      0.72      0.75       379

    accuracy                           0.68      1002
   macro avg       0.68      0.67      0.67      1002
weighted avg       0.68      0.68      0.68      1002



(0.6776447105788423,
 0.6839785813793443,
 0.6776447105788423,
 0.6790486993469321)

**(4)  SGDClassifier used “one versus all” (OVA) strategy to handle multi-class strategy. For each of K classes, the binary classifier will try to predict that class and the rest other K-1 classes.**

In [43]:
# Printing features with highest weights.
best_SGD = SGDClassifier(loss = 'log', penalty='l2',  shuffle=True, alpha=0.0091).fit(X_train, y_train)
y_pred = best_SGD.predict(X_test)
weights = best_SGD.coef_
weights

array([[ 0.00642179, -0.03722184, -0.02245802, ..., -0.00130787,
        -0.00533004,  0.00484716],
       [ 0.04752901, -0.03313101, -0.03502409, ..., -0.00481813,
        -0.00118829, -0.00603774],
       [-0.05149932,  0.05194289,  0.0588562 , ...,  0.00730511,
         0.00558568, -0.00077458]])

In [48]:
features = vectorizer.get_feature_names()

feature_weights = {}
for i in range(len(weights)):
  print('\n==============================================')
  print ('\nclass: {}'.format(i+1))
  for w in range(len(weights[i])):
     feature_weights[features[w]] = np.abs(weights[i][w])
  # Sorting the dictionary in descending order
  sorted_feature_weights = {k:v for k, v in sorted(feature_weights.items(), key = lambda item: item[1], reverse=True)}

  # Print the weights learned for each class
  print('10 most important features (in descending order): ')
  for k, v in list(sorted_feature_weights.items())[:20]:
    print ('{}: {:.5f}'. format(k,v))




class: 1
10 most important features (in descending order): 
bad: 0.38316
fine: 0.35764
great: 0.33347
boring: 0.31654
ridiculous: 0.30678
yet: 0.28239
quite: 0.27073
satisfying: 0.26932
was: 0.26293
keeps: 0.26146
although: 0.25424
dull: 0.25004
awful: 0.24614
only: 0.24473
best: 0.24130
failed: 0.23995
lifeless: 0.23638
poorly: 0.23506
wrong: 0.23429
many: 0.23244


class: 2
10 most important features (in descending order): 
unfortunately: 0.42315
problems: 0.32225
problem: 0.31881
interesting: 0.30740
nice: 0.30036
difficult: 0.28090
perfect: 0.27811
moments: 0.27140
ever: 0.26876
ve: 0.26770
able: 0.24280
albeit: 0.23800
cute: 0.23743
not: 0.23061
thumbs: 0.22704
single: 0.22302
bizarre: 0.22116
though: 0.21565
feels: 0.21454
particularly: 0.21433


class: 3
10 most important features (in descending order): 
unfortunately: 0.56859
perfect: 0.41944
great: 0.38152
problem: 0.37956
interesting: 0.37337
bad: 0.33584
flat: 0.32449
nice: 0.30220
predictable: 0.29863
too: 0.29401
seemed:

**Q6. How does the hyperplane of the classifier look like?**

In [56]:
# import seaborn as sns
# sns.displot(weights)
hyperplane = best_SGD.decision_function(X_train).T
sns.displot(np.flatnonzero(hyperplane))

array([[-0.54920639, -1.67291965, -4.20045942, ..., -2.32302828,
        -2.49683199, -3.04083907],
       [ 0.19182483,  0.62629799, -0.98777304, ..., -0.64845529,
        -1.26332743, -2.10099883],
       [-1.87374769, -1.73727737,  1.56284437, ...,  0.24830053,
         1.00273638,  1.80255085]])